<a href="https://colab.research.google.com/github/peculab/PythonAI4Beginners/blob/main/week13_%E7%B6%B2%E8%B7%AF%E8%B3%87%E6%96%99%E8%87%AA%E5%8B%95%E7%88%AC%E5%8F%96%E8%88%87%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

✅ 1. 從網路上抓取新聞（Web Scraping）

以 ptt 為例，使用 requests + BeautifulSoup 抓取新聞標題與內容。

✅ 2. 分析新聞內容（NLP 分析）

用 SnowNLP（中文）進行情緒分析。

✅ 3. 視覺化分析結果

用 pandas 做表格處理，plotly 視覺化結果。

In [1]:
!pip install snownlp plotly beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 28.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=87e63f5de434ad5389e7998d63e3e221ed016237f8b45752dc2923bfac8ee396
  Stored in directory: /root/.cache/pip/wheels/8a/0a/37/f15b8568f5463f1427466f701e9d3ba514035eb703f885efee
Successfully built snownlp


In [2]:
import requests
from bs4 import BeautifulSoup
from snownlp import SnowNLP
import pandas as pd
import plotly.express as px

# 模擬 header 與 cookies（解鎖 Gossiping）
HEADERS = {
    'User-Agent': 'Mozilla/5.0'
}
COOKIES = {'over18': '1'}

In [3]:
def get_ptt_articles(board='Stock', max_pages=2):
    base_url = f'https://www.ptt.cc/bbs/{board}/index.html'
    articles = []

    for _ in range(max_pages):
        res = requests.get(base_url, headers=HEADERS, cookies=COOKIES)
        soup = BeautifulSoup(res.text, 'html.parser')
        entries = soup.select('div.r-ent')

        for entry in entries:
            title_tag = entry.select_one('div.title > a')
            if title_tag:
                title = title_tag.text.strip()
                link = 'https://www.ptt.cc' + title_tag['href']

                try:
                    article_res = requests.get(link, headers=HEADERS, cookies=COOKIES)
                    article_soup = BeautifulSoup(article_res.text, 'html.parser')
                    content = article_soup.select_one('#main-content').text.split('--')[0]
                    content = '\n'.join(line for line in content.split('\n') if not line.startswith('※'))
                    sentiment = SnowNLP(content).sentiments
                except Exception as e:
                    content = ""
                    sentiment = None

                articles.append({
                    '標題': title,
                    '連結': link,
                    '情緒分數': sentiment
                })

        # 下一頁連結
        paging = soup.select_one('div.btn-group-paging > a.btn.wide:nth-child(2)')
        if paging:
            base_url = 'https://www.ptt.cc' + paging['href']
        else:
            break

    return pd.DataFrame(articles)

In [4]:
# 抓 PTT Stock 看板 2 頁
df = get_ptt_articles('Stock', max_pages=2)
df = df.dropna(subset=['情緒分數'])

In [5]:
df

,標題,連結,情緒分數
0,Re: [請益] 為何 nv 財報亮眼還會大跌阿,https://www.ptt.cc/bbs/Stock/M.1763671803.A.D9...,4.853018e-06
1,Re: [請益] 為何 nv 財報亮眼還會大跌阿,https://www.ptt.cc/bbs/Stock/M.1763673472.A.44...,2.097918e-06
2,[請益] AI持續發展是否會引起就業市場長期蕭條,https://www.ptt.cc/bbs/Stock/M.1763676912.A.F9...,7.633894e-12
3,[標的] SOXS.US 三倍做空半導體,https://www.ptt.cc/bbs/Stock/M.1763680459.A.E4...,8.881784e-16
4,[心得] 現在跑不如好好待著,https://www.ptt.cc/bbs/Stock/M.1763681278.A.7D...,1.344917e-08
6,Re: [請益] 美股越來越有2021的味道了？,https://www.ptt.cc/bbs/Stock/M.1763682530.A.5A...,0.000000e+00
7,[新聞] 鴻海與OpenAI合作細節曝 在美國打造下一,https://www.ptt.cc/bbs/Stock/M.1763682989.A.DC...,4.694584e-01
8,[請益] 持有現金如何克制下單的衝動,https://www.ptt.cc/bbs/Stock/M.1763684414.A.AA...,4.042287e-09
9,Re: [新聞] 力積電基本面優異 營運加速點火熱到明年,https://www.ptt.cc/bbs/Stock/M.1763684438.A.3F...,0.000000e+00
10,Re: [新聞] 鴻海與OpenAI合作細節曝 在美國打造下一,https://www.ptt.cc/bbs/Stock/M.1763684444.A.1E...,1.223732e-11


In [6]:
# 畫圖
fig = px.bar(
    df.sort_values('情緒分數'),
    x='情緒分數',
    y='標題',
    orientation='h',
    title='PTT 文章情緒分析',
    height=600
)
fig.update_layout(yaxis=dict(automargin=True))
fig.show()

In [7]:
import jieba
from collections import Counter
import jieba.analyse

/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning:

invalid escape sequence '\.'

/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning:

invalid escape sequence '\s'

/usr/local/lib/python3.12/dist-packages/jieba/finalseg/__init__.py:78: SyntaxWarning:

invalid escape sequence '\.'

/usr/local/lib/python3.12/dist-packages/jieba/posseg/__init__.py:16: SyntaxWarning:

invalid escape sequence '\.'

/usr/local/lib/python3.12/dist-packages/jieba/posseg/__init__.py:17: SyntaxWarning:

invalid escape sequence '\.'

/usr/local/lib/python3.12/dist-packages/jieba/posseg/__init__.py:18: SyntaxWarning:

invalid escape sequence '\s'

/usr/local/lib/python3.12/dist-packages/jieba/posseg/__init__.py:21: SyntaxWarning:

invalid escape sequence '\.'



In [19]:
# 把所有標題串在一起當成一段文字
text = ''.join(df['標題'].astype(str).tolist())

In [20]:
# 避免輸出有 \n 跟空格
text = text.replace("\n", "").strip()

In [21]:
# 停用詞
stop_words = set([
    '所以', '好', '因為', '；', '的', '是', '了', '欲', '也', '在', '和', '就', '不', '有',
    '以', '與', '、', '為', '這個', '而', '「', '」', '，', '。', '《', '》', '對', '於', '\n', ' '
])

In [22]:
# jieba 中添加自定義詞彙
custom_dict = [
    '美股',
    '鴻海'
]

In [23]:
for word in custom_dict:
    jieba.add_word(word)

In [24]:
# 先斷詞，這裡才會得到 words（修正點）
words = [w for w in jieba.cut(text) if w.strip()]

In [25]:
# 過濾停用詞
filtered_words = [w for w in words if w not in stop_words]

In [26]:
# 統計詞頻
word_count = Counter(filtered_words)

In [27]:
# 只保留出現次數 > 1 的詞
filtered_word_count = {
    w: c for w, c in word_count.items() if c > 1
}

In [28]:
print("詞頻統計（出現次數 > 1）：")
for w, c in sorted(filtered_word_count.items(), key=lambda x: x[1], reverse=True):
    print(w, c)

詞頻統計（出現次數 > 1）：
[ 43
] 43
: 14
Re 13
請益 13
新聞 12
為何 6
？ 6
/ 6
nv 5
財報 5
亮眼 5
還會 5
大跌 5
阿 5
標 5
心得 5
現在 4
美股 4
閒聊 4
跑 3
不如 3
好好 3
待 3
著 3
越來 3
越 3
2021 3
味道 3
細節 3
一 3
公告 3
4 3
AI 2
做空 2
鴻海 2
OpenAI 2
合作 2
曝 2
美國 2
打造 2
下 2
營運 2
台積電 2
2025 2
11 2
21 2
盤中 2
： 2
股票 2
( 2
) 2
- 2
情報 2


In [29]:
# 用 TF-IDF 抓關鍵字（修正 keywords 未定義）
keywords = jieba.analyse.extract_tags(text, topK=20)
print("\n關鍵字：", keywords)

# 這邊就直接把「每一則標題」當作一句話
sentences = df['標題'].astype(str).tolist()

# 依照「含有關鍵字的多寡」來排序句子，取前三句當簡易摘要
top_sentences = sorted(
    sentences,
    key=lambda s: sum(1 for w in jieba.cut(s) if w in keywords),
    reverse=True
)[:3]

summary = '。'.join(top_sentences).replace("\n", "")
print("\n簡易摘要：", summary)


關鍵字： ['Re', '請益', '新聞', '為何', 'nv', '財報', '還會', '亮眼', '現在', '美股', '閒聊', '心得', '越來', '2021', '細節', '大跌', 'AI', '做空', '鴻海', 'OpenAI']

簡易摘要： Re: [請益] 為何 nv 財報亮眼還會大跌阿。Re: [請益] 為何 nv 財報亮眼還會大跌阿。Re: [請益] 為何 nv 財報亮眼還會大跌阿
